<a href="https://colab.research.google.com/github/kuock0129/DDPM-Denoising-Diffusion-Probabilistic-Model/blob/main/StableDiffusionWebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Torch version: 2.6.0+cu124
CUDA available: False
Device: CPU


In [ ]:
# Install with CUDA 12.1 for T4
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
# 3. Clone WebUI repo
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/webui
%cd /content/webui

Cloning into '/content/webui'...
remote: Enumerating objects: 34945, done.
remote: Total 34945 (delta 0), reused 0 (delta 0), pack-reused 34945 (from 1)
Receiving objects: 100% (34945/34945), 35.53 MiB | 28.14 MiB/s, done.
Resolving deltas: 100% (24409/24409), done.
/content/webui


In [ ]:
# Clone ControlNet Extension
!git clone https://github.com/Mikubill/sd-webui-controlnet extensions/sd-webui-controlnet

Cloning into 'extensions/sd-webui-controlnet'...
remote: Enumerating objects: 9979, done.
remote: Counting objects: 100% (724/724), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 9979 (delta 669), reused 625 (delta 625), pack-reused 9255 (from 2)
Receiving objects: 100% (9979/9979), 18.11 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (5980/5980), done.


In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
import os
os.environ["COMMANDLINE_ARGS"] = " --enable-insecure-extension-access --skip-torch-cuda-test --share"

In [ ]:
!apt-get install -y aria2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
# SD v1.5
!mkdir -p models/Stable-diffusion
!aria2c --dir=models/Stable-diffusion --out=v1-5-pruned-emaonly.safetensors https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors


04/11 07:22:34 [NOTICE] Downloading 1 item(s)

04/11 07:22:34 [NOTICE] CUID#7 - Redirecting to https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors

04/11 07:22:34 [NOTICE] CUID#7 - Redirecting to https://cdn-lfs.hf.co/repos/66/6f/666f465fa70158515404e8de2c6bc6fe2f90c46f9296293aa14daededeb32c52/6ce0161689b3853acaa03779ec93eafe75a02f4ced659bee03f50797806fa2fa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27v1-5-pruned-emaonly.safetensors%3B+filename%3D%22v1-5-pruned-emaonly.safetensors%22%3B&Expires=1744358261&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDM1ODI2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82Ni82Zi82NjZmNDY1ZmE3MDE1ODUxNTQwNGU4ZGUyYzZiYzZmZTJmOTBjNDZmOTI5NjI5M2FhMTRkYWVkZWRlYjMyYzUyLzZjZTAxNjE2ODliMzg1M2FjYWEwMzc3OWVjOTNlYWZlNzVhMDJmNGNlZDY1OWJlZTAzZjUwNzk3ODA2ZmEyZmE%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=v8EoVD-zrz2

In [ ]:
# ControlNet - OpenPose
!mkdir -p models/ControlNet
!aria2c --dir=models/ControlNet --out=control_v11p_sd15_openpose.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth


04/11 07:23:25 [NOTICE] Downloading 1 item(s)

04/11 07:23:25 [NOTICE] CUID#7 - Redirecting to https://cdn-lfs.hf.co/repos/e7/4c/e74c9ddc88b5a293fa8aafeb615d6c61cb3aaa34dc7ce9e25100dfae2b09bf55/db97becd92cd19aff71352a60e93c2508decba3dee64f01f686727b9b406a9dd?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27control_v11p_sd15_openpose.pth%3B+filename%3D%22control_v11p_sd15_openpose.pth%22%3B&Expires=1744359805&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NDM1OTgwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9lNy80Yy9lNzRjOWRkYzg4YjVhMjkzZmE4YWFmZWI2MTVkNmM2MWNiM2FhYTM0ZGM3Y2U5ZTI1MTAwZGZhZTJiMDliZjU1L2RiOTdiZWNkOTJjZDE5YWZmNzEzNTJhNjBlOTNjMjUwOGRlY2JhM2RlZTY0ZjAxZjY4NjcyN2I5YjQwNmE5ZGQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=tNv-ZVchDlQSy2cIiba6OSU0zzo0qcGBguPP3kvYR4DOqAqDylFshdueRM40HNrPlkkkO1n1SnvQFcfw3%7EpzZl1vHrfFeQii9qe9whJ10XQj-cy9J%7EJWm4ZWL0WGdLiVeU7BEybi7UxBZCDEMTPLso8xA-L8i3m9MPj2nX9414

In [ ]:
!cd stable-diffusion-webui/extensions
!git clone https://github.com/continue-revolution/sd-webui-animatediff

/bin/bash: line 1: cd: stable-diffusion-webui/extensions: No such file or directory
Cloning into 'sd-webui-animatediff'...
remote: Enumerating objects: 1136, done.
remote: Counting objects: 100% (697/697), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 1136 (delta 591), reused 515 (delta 515), pack-reused 439 (from 2)
Receiving objects: 100% (1136/1136), 406.45 KiB | 4.73 MiB/s, done.
Resolving deltas: 100% (722/722), done.


In [ ]:
!wget https://huggingface.co/guoyww/animatediff-motion-models/resolve/main/mm_sd_v14.ckpt -O /content/stable-diffusion-webui/extensions/sd-webui-animatediff/model/Motion_Module/mm_sd_v14.ckpt

/content/stable-diffusion-webui/extensions/sd-webui-animatediff/model/Motion_Module/mm_sd_v14.ckpt: No such file or directory


In [ ]:
!python launch.py


Python 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Version: v1.10.1
Commit hash: 82a973c04367123ae98bd9abdf80d9eda9b910e2
ControlNet init warning: Unable to install insightface automatically. Please try run `pip install insightface` manually.
Launching Web UI with arguments: --enable-insecure-extension-access --skip-torch-cuda-test --share
2025-04-11 07:50:03.518590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744357803.561315    9749 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744357803.573855    9749 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
no module 'xformers'. Processing without...
no module 'xformers'. Processing without...


In [ ]:
#@title 🚀 Launch AUTOMATIC1111 WebUI + ControlNet (with character consistency setup)
!apt -y update -qq && apt -y install -qq aria2 git
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/webui
%cd /content/webui

# Add ControlNet extension
!git clone https://github.com/Mikubill/sd-webui-controlnet extensions/sd-webui-controlnet

# Download SD1.5 checkpoint
!mkdir -p /content/webui/models/Stable-diffusion
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors \
  -d /content/webui/models/Stable-diffusion -o v1-5-pruned-emaonly.safetensors

# Download ControlNet models (OpenPose as example)
!mkdir -p /content/webui/extensions/sd-webui-controlnet/models
!aria2c -c -x 16 -s 16 -k 1M \
  https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth \
  -d /content/webui/extensions/sd-webui-controlnet/models



32 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
git is already the newest version (1:2.34.1-1ubuntu1.12).
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1